DONE - UPDATED 29 AUG 25

# 02_NB25 Sample Extraction

In [1]:
# checks conda path and environment
import pandas as pd
import os
from datetime import timedelta
import sys
print(sys.executable)

/home/jupyter-tommills/.conda/envs/nepalbirds25/bin/python


In [2]:
os.getcwd()

'/home/jupyter-tommills/Nepal_Birds_TM'

***************

## **Reduce sample size to stratified sample**

**Aim:** 
Subset 2021 feature set to:
- Recorders with at least 7 days continuous recording 
- Take every 8th 1-min file per recorder; ie 1-min every 2 hours 

In [3]:
# read in datasets 
nepalbirds_metadata2021 = pd.read_csv("data/nepal/metadata/nepalbirds_metadata2021.csv")
nepalbirds_features = pd.read_parquet("data/nepal/birdnet_features/nepalfeatures_merged.parquet")

In [4]:
# filter features by year = 2021
nepalbirds_features['year'] = nepalbirds_features['path'].str.split('/').str[4]
nepalbirds_features2021 = nepalbirds_features[nepalbirds_features['year'] == '2021']

### **Step 1 - Add times, regimes and site columns to the features dataset**

In [5]:
# add 'recorded_on' and 'site' as columns
NB25_fm_2021 = nepalbirds_features2021.merge(
    nepalbirds_metadata2021[['path', 'recorded_on', 'site']],
    on='path',
    how='left')
# add 'regime' column
NB25_fm_2021['regime'] = NB25_fm_2021['site'].str[:2]

In [6]:
# check correct number of sites
unique_sites = NB25_fm_2021['site'].nunique()
print(f"Unique sites: {unique_sites}")

Unique sites: 58


### **Step 2 - Filter out sites with fewer than 7 days audio**

In [7]:
nepalbirds_features2021.head()

,path,start_time,end_time,BirdNET_0,BirdNET_1,BirdNET_2,BirdNET_3,BirdNET_4,BirdNET_5,BirdNET_6,...,BirdNET_1015,BirdNET_1016,BirdNET_1017,BirdNET_1018,BirdNET_1019,BirdNET_1020,BirdNET_1021,BirdNET_1022,BirdNET_1023,year
3423,downloads/NP/NP30/PAM/2021/NP30_20210422_14300...,0.0,3.0,0.091834,0.249815,0.113273,1.078636,0.365445,0.370891,0.000000,...,1.012715,0.219605,0.0,0.231928,0.198208,0.433792,0.138966,0.217478,0.581902,2021
3424,downloads/NP/NP30/PAM/2021/NP30_20210422_14300...,3.0,6.0,0.195594,0.063544,0.079855,0.779105,0.266044,0.329836,0.001157,...,0.927807,0.000000,0.0,0.649267,0.114159,0.270783,0.031773,0.259857,0.472207,2021
3425,downloads/NP/NP30/PAM/2021/NP30_20210422_14300...,6.0,9.0,0.133215,0.186157,0.000000,0.993188,0.253370,0.284816,0.000000,...,0.968302,0.132490,0.0,0.496252,0.000000,0.413925,0.014390,0.317584,0.333687,2021
3426,downloads/NP/NP30/PAM/2021/NP30_20210422_14300...,9.0,12.0,0.069973,0.051973,0.785590,0.129056,0.289650,0.508400,0.000000,...,0.207206,0.000000,0.0,0.265115,0.000000,0.369664,0.232464,0.447781,0.296311,2021
3427,downloads/NP/NP30/PAM/2021/NP30_20210422_14300...,12.0,15.0,0.178235,0.204697,0.000000,0.599773,0.222770,0.436152,0.058033,...,1.077979,0.101276,0.0,0.362332,0.060755,0.163080,0.113382,0.345325,0.291305,2021


In [8]:
# read in dataset of sites with 7 days or more data
sites_7daysplus = pd.read_csv("sites_7daysplus.csv")
sites_7daysplus.head()

,site,min,max,days_span,regime
0,OBZ39,2021-04-23 17:45:00,2021-05-02 13:45:00,8,OB
1,OBZ07,2021-04-23 19:00:00,2021-05-02 18:00:00,8,OB
2,BZ40,2021-04-24 12:15:00,2021-05-03 16:45:00,9,BZ
3,OBZ35,2021-04-22 16:00:00,2021-05-02 13:00:00,9,OB
4,BZ48,2021-03-26 14:15:00,2021-04-09 20:30:00,14,BZ


In [9]:
# filter to only include sites from nepalbirds_features2021
NB25_fm_2021_7day = NB25_fm_2021[
    NB25_fm_2021['site'].isin(sites_7daysplus['site'])]

# count sites in new dataframe to check
site_count_2021 = NB25_fm_2021['site'].nunique()
site_count_2021_7days = NB25_fm_2021_7day['site'].nunique()
print(f"Sites: full = {site_count_2021}, filtered = {site_count_2021_7days}")
print("Correct")

Sites: full = 58, filtered = 58
Correct


In [10]:
#count per regime
regime_counts = sites_7daysplus.groupby('regime').size().reset_index(name='count')
print(regime_counts)

  regime  count
0     BZ     20
1     NP     17
2     OB     21


### **Step 3 - limit data to first 7 days**

In [13]:
# refine the dataset

#select only necessary columns from sites_7daysplus
summary_cols = ['site', 'min', 'max']
summary_trimmed = sites_7daysplus[summary_cols].copy()

# check for key integrity
assert summary_trimmed['site'].is_unique, "Non-unique site keys in site_summary_2021"

# merge with features plus embeddings dataframe
features_enriched = NB25_fm_2021_7day.merge(
    summary_trimmed,
    on='site',
    how='left',
    validate='many_to_one')

In [14]:
# filter to first 7 days for each site

# ensure datetime formats
features_enriched['recorded_on'] = pd.to_datetime(features_enriched['recorded_on'], errors='coerce')
features_enriched['min'] = pd.to_datetime(features_enriched['min'], errors='coerce')

# apply vector filter
mask = (
    (features_enriched['recorded_on'] >= features_enriched['min']) &
    (features_enriched['recorded_on'] <= features_enriched['min'] + pd.Timedelta(days=7)))

# filter dataframe
nepalbirds_features2021_7daylimit = features_enriched[mask]

In [17]:
nepalbirds_features2021_7daylimit.shape

(774174, 1033)

### **Step 4 - filter to every 1/8 recordings to make processing manageable**

In [29]:
# create a function to return only rows recorded on the hour for every even hour with copy to avoid warnings
def filter_every_other_hour_on_the_hour(df):
    df = df.copy() 
    df['recorded_on'] = pd.to_datetime(df['recorded_on'], errors='coerce')
    on_the_hour = df[
        (df['recorded_on'].dt.minute == 0) &
        (df['recorded_on'].dt.second == 0)]

# adjust to keep only rows where the hour is even
    filtered = on_the_hour[on_the_hour['recorded_on'].dt.hour % 2 == 0]
    return filtered.sort_values(['site', 'recorded_on']).reset_index(drop=True)

# apply to your features dataframe
nepalbirds2021_sample3_even = filter_every_other_hour_on_the_hour(nepalbirds_features2021_7daylimit)

In [30]:
nepalbirds2021_sample3_even.shape

(96807, 1033)

**Export stratified 2021 sample to csv**

In [31]:
nepalbirds2021_sample3_even.to_csv("nepalbirds2021_sample3_even.csv", index=False)

***************